# 100 Sports Image Classification

## データのインポート

In [ ]:
# 初回のみ実行

# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("gpiosenka/sports-classification")

# print("Path to dataset files:", path)

## 設定値

In [ ]:
EPOCHS = 50 # エポック数
IMG_SIZE = 64 # 画像のサイズ
BATCH_SIZE = 32 # バッチサイズ
FILTERS_SIZE = 32 # フィルタサイズ

## データの表示

In [ ]:
import pandas as pd

data_path = './kagglehub_cache/datasets/gpiosenka/sports-classification/versions/9/'
csv_path = data_path + 'sports.csv'
df = pd.read_csv(csv_path)
df.head()

### カテゴリ列のユニーク値チェック

In [ ]:
# categorical_columns = ["labels",'data set']
categorical_columns = ['data set']
for col in categorical_columns:
    if col in df.columns:
        unique_values = df[col].value_counts()
        print(f"{col} 列のユニーク値 ({len(unique_values)} 個):")
        for value, count in unique_values.items():
            print(f"  {value}: {count} 件")

## データの分離

In [ ]:
df_train = df[df['data set'] == 'train']
df_test = df[df['data set'] == 'test']
df_valid = df[df['data set'] == 'valid']

display(df_train.head())
# display(df_test.head())
# display(df_valid.head())

## CNN

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### データの前処理

#### 訓練用データセットの前処理

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
train_set = train_datagen.flow_from_directory(data_path + 'train',
                                                 target_size = (IMG_SIZE, IMG_SIZE),
                                                 batch_size = BATCH_SIZE,
                                                 class_mode = 'categorical')
# 2分類の場合は class_mode = 'binary' を指定
# 多分類の場合は class_mode = 'categorical' を指定

#### 検証用データセットの前処理

In [ ]:
valid_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
valid_set = valid_datagen.flow_from_directory(data_path + 'valid',
                                                 target_size = (IMG_SIZE, IMG_SIZE),
                                                 batch_size = BATCH_SIZE,
                                                 class_mode = 'categorical')

#### テストデータセットの前処理

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
test_set = test_datagen.flow_from_directory(data_path + 'test',
                                                 target_size = (IMG_SIZE, IMG_SIZE),
                                                 batch_size = BATCH_SIZE,
                                                 class_mode = 'categorical')

### CNNの構築

#### イニシャライズ

In [ ]:
cnn = tf.keras.models.Sequential()

#### 畳み込みandプーリング

In [ ]:
# 一層目
cnn.add(tf.keras.layers.Conv2D(filters=FILTERS_SIZE, kernel_size=3, activation='relu', input_shape=[IMG_SIZE, IMG_SIZE, 3]))
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))

# 二層目
cnn.add(tf.keras.layers.Conv2D(filters=FILTERS_SIZE, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))

#### Flattening

In [ ]:
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

#### 出力層の追加

In [ ]:
cnn.add(tf.keras.layers.Dense(units=100, activation='softmax'))
# 2分類の場合は units=1, activation='sigmoid' を指定
# 多分類の場合は units=[number], activation='softmax' を指定

### モデル学習

#### モデルのコンパイルと訓練

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
# 2分類の場合は loss = 'binary_crossentropy' を指定
# 多分類の場合は loss = 'categorical_crossentropy' を指定

#### モデルの訓練

In [ ]:
cnn.fit(x = train_set, validation_data = valid_set, epochs = EPOCHS)

## 結果の出力

In [ ]:
import os
import json

# クラス名のマッピング（index → label）
class_indices = train_set.class_indices

# 保存用ディレクトリ
model_path = './ml'
os.makedirs(model_path, exist_ok=True)

# class_indices を JSON に保存（例: { 'air hockey': 0, 'archery': 1, ... }）
with open(os.path.join(model_path, "class_indices.json"), "w") as f:
    json.dump(class_indices, f, indent=2, ensure_ascii=False)
print("クラスインデックス（class_indices）を JSON 出力しました。")

# モデル保存
model_filename = f'model_{EPOCHS}epochs.h5'
cnn.save(os.path.join(model_path, model_filename))
print(f"モデルを保存しました: {model_filename}")